In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
import mne
import pathlib
import pytorch_lightning as pl
import torch
import torcheeg
import xgboost
import wandb

### Interface with data

In [3]:
data_dir = pathlib.Path("/data0/practical-sose23/brain-age/data/bap/")
assert data_dir.is_dir()

data_dir_healthy = data_dir / "healthy_controls/preprocessed"
assert data_dir_healthy.is_dir()

data_dir_chronic_pain = data_dir / "chronic_pain_patients"
assert data_dir_chronic_pain.is_dir()

In [4]:
eeg_files_all = list(data_dir.rglob("*.vhdr"))

eeg_files_raw = [f for f in eeg_files_all if not "preprocessed" in str(f)]
eeg_files_preprocessed = [f for f in eeg_files_all if "preprocessed" in str(f)]

eeg_files_healthy = list(data_dir_healthy.rglob("*.vhdr"))
eeg_files_chronic_pain = [f for f in eeg_files_preprocessed if "chronic_pain_patients" in str(f)]


### Analyze the subject data

In [5]:
def get_subj_id(eeg_file: pathlib.Path):
    return eeg_file.stem.split("_")[-3]

def get_age(df_subj, eeg_file: pathlib.Path):
    subj_id = get_subj_id(eeg_file)
    return df_subj[df_subj["Subject ID"] == subj_id]["Age(years)"].values[0]

#### Load clinical data

In [6]:
f_subj = data_dir / "clinical_data_updated_2020-08-04.ods"
assert f_subj.is_file()

In [7]:
df_subj_chronic_pain = pd.read_excel(f_subj, sheet_name=0, engine="odf", skiprows=1)
df_subj_healthy = pd.read_excel(f_subj, sheet_name=1, engine="odf", skiprows=0)

df_subj = pd.concat([df_subj_chronic_pain, df_subj_healthy])
df_subj["Age(years)"] = df_subj["Age(years)"].fillna(df_subj["Age (years)"])

print(f"# recorded subjects:      {len(df_subj)}")
print(f"# raw eeg files:          {len(eeg_files_raw)}")
print(f"# preprocessed eeg files: {len(eeg_files_preprocessed)}")

zfill_ints = lambda x:str(x).zfill(3) if type(x) else x
df_subj_chronic_pain["Subject ID"] = df_subj_chronic_pain["Subject ID"].astype(str).map(zfill_ints)
df_subj_healthy["Subject ID"] = df_subj_healthy["Subject ID"].astype(str).map(zfill_ints)
df_subj["Subject ID"] = df_subj["Subject ID"].astype(str).map(zfill_ints)

df_subj

# recorded subjects:      189
# raw eeg files:          190
# preprocessed eeg files: 189


,Subject ID,Project prefix,Age(years),Sex (m/f),Pain duration(months),Avg. painIntensity (0 – 10),Curr. Pain Intensity(0 – 10),BDI,PDQ,MQS,VR-12 PCS,VR-12 MCS,PDI,Age (years)
0,VP02,chronic_back_pain,67.0,m,360.0,5.0,5.0,3.0,28.0,4.0,NaN,NaN,NaN,NaN
1,VP04,chronic_back_pain,54.0,f,120.0,5.0,4.0,35.0,14.0,31.7,NaN,NaN,NaN,NaN
2,VP05,chronic_back_pain,64.0,f,96.0,7.0,5.0,7.0,16.0,11.4,NaN,NaN,NaN,NaN
3,VP07,chronic_back_pain,41.0,m,22.0,6.0,7.0,15.0,22.0,6.1,NaN,NaN,NaN,NaN
4,VP08,chronic_back_pain,72.0,m,480.0,8.0,8.0,11.0,10.0,8.4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,vpH64,various,79.0,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0
84,vpH66,various,56.0,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.0
85,vpH67,various,54.0,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.0
86,vpH68,various,55.0,f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0


#### Check the metadata and set channel types

In [8]:
example_raw = mne.io.read_raw_brainvision(eeg_files_chronic_pain[0])
example_raw

Extracting parameters from /data0/practical-sose23/brain-age/data/bap/chronic_pain_patients/chronic_back_pain/preprocessed/Exp_eyes_closed_vp11_eyes_closed.vhdr...
Setting channel info structure...


/tmp/ipykernel_431469/2177493425.py:1: RuntimeWarning: No coordinate information found for channels ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1', 'P2', 'C1', 'C2', 'FT9', 'FT10', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'PO9', 'Iz', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'PO10', 'CPz', 'POz']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  example_raw = mne.io.read_raw_brainvision(eeg_files_chronic_pain[0])
/tmp/ipykernel_431469/2177493425.py:1: RuntimeWarning: Not setting positions of 64 misc channels found in montage:
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P

<RawBrainVision | Exp_eyes_closed_vp11_eyes_closed.dat, 65 x 300004 (300.0 s), ~71 kB, data not loaded>

In [9]:
channel_to_channel_type = {ch_name:"eeg" for ch_name in example_raw.ch_names}
channel_to_channel_type.update({"LE":"misc", "RE":"misc"})
eeg_chs = [ch for ch in example_raw.ch_names if not (ch=="RE" or ch=="LE")]
example_raw.set_channel_types(channel_to_channel_type)


/tmp/ipykernel_431469/179859254.py:4: RuntimeWarning: The unit for channel(s) AF3, AF4, C1, C2, C3, C4, C5, C6, CP1, CP2, CP3, CP4, CP5, CP6, CPz, Cz, F3, F4, F5, F6, F7, F8, FC1, FC2, FC3, FC4, FC5, FC6, FT10, FT7, FT8, FT9, Fp1, Fp2, Fpz, Fz, Iz, O1, O2, Oz, P1, P2, P3, P4, P5, P6, P7, P8, PO10, PO3, PO4, PO7, PO8, PO9, POz, Pz, T7, T8, TP10, TP7, TP8, TP9 has changed from NA to V.
  example_raw.set_channel_types(channel_to_channel_type)


<RawBrainVision | Exp_eyes_closed_vp11_eyes_closed.dat, 65 x 300004 (300.0 s), ~71 kB, data not loaded>

In [10]:
montage = mne.channels.make_standard_montage("standard_1020")
example_raw = example_raw.set_montage(montage)

In [11]:
for f in eeg_files_preprocessed:
    if not f.is_file():
        print(f)

### Extract PSD dataset

In [12]:
ages = []
psds = []

for f in eeg_files_preprocessed:
    
    age = get_age(df_subj, f)
    raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)
    raw = raw.set_channel_types(channel_to_channel_type, verbose=False)
    raw = raw.crop(raw.tmin+30, raw.tmax-30)
    
    raw = raw.notch_filter(freqs=50, notch_widths=0.5)
    raw = raw.filter(l_freq=1, h_freq=100)
    
    epochs = mne.make_fixed_length_epochs(raw, duration=10, preload=True)
    epochs_psds = np.log(epochs.compute_psd("welch", fmin=1, fmax=100, n_fft=int(5*1024)))
    
    psds.append(epochs_psds)
    ages.append(len(epochs_psds)*[age])



/tmp/ipykernel_431469/1559163461.py:7: RuntimeWarning: No coordinate information found for channels ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1', 'P2', 'C1', 'C2', 'FT9', 'FT10', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'PO9', 'Iz', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'PO10', 'CPz', 'POz']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)


Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.25
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.00 Hz)
- Upper passband edge: 50.75 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 51.00 Hz)
- Filter length: 6601 samples (6.601 sec)



/tmp/ipykernel_431469/1559163461.py:7: RuntimeWarning: Not setting positions of 64 misc channels found in montage:
['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'Oz', 'FC1', 'FC2', 'CP1', 'CP2', 'FC5', 'FC6', 'CP5', 'CP6', 'TP9', 'TP10', 'LE', 'RE', 'P1', 'P2', 'C1', 'C2', 'FT9', 'FT10', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'C5', 'C6', 'P5', 'P6', 'PO9', 'Iz', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'Fpz', 'PO10', 'CPz', 'POz']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(f, verbose=False, preload=True)
/tmp/ipykernel_431469/1559163461.py:8: RuntimeWarning: The unit for channel(s) AF3, AF4, C1, C2, C3, C4, C5, C6, CP1, CP2, CP3, CP4, CP5, CP6, CPz, Cz, F3, F4, F5, F6, F7, F8, FC1, FC2, FC3, FC4, FC5, FC6, FT10, FT7

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 sec)



[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


...

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 3301 samples (3.301 sec)



[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
41 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 41 events and 10000 original time points ...
1 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.6s finished


Effective window size : 5.120 (s)


In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold, cross_val_score, KFold, train_test_split
from sklearn.pipeline import Pipeline

### Baselines with train-validation split

In [17]:
ch_name_to_ch_pos = montage.get_positions()["ch_pos"]

pos = [ch_name_to_ch_pos[ch_name] for ch_name in example_raw.ch_names if channel_to_channel_type[ch_name]=="eeg"]
pos = np.stack(pos)

In [18]:
### small sanity check: shuffling retains relationships between labels and features

permutation1 = np.arange(0, 10)
permutation2 = np.arange(0, 10)
permutation3 = np.array([3*[i] for i in range(10)])

np.random.seed(42)
np.random.shuffle(permutation1)
np.random.seed(42)
np.random.shuffle(permutation2)
np.random.seed(42)
np.random.shuffle(permutation3)

permutation1[0], permutation2[0], permutation3[0]

(8, 8, array([8, 8, 8]))

In [15]:
np.random.seed(42)
np.random.shuffle(psds)
np.random.seed(42)
np.random.shuffle(ages)

n_train = int(0.7*len(psds))
n_val = int(0.15*len(psds))
n_test = int(0.15*len(psds))

psds_train = np.concatenate(psds[:n_train])
psds_train = psds_train.reshape(len(psds_train), -1)
ages_train = np.concatenate(ages[:n_train])

psds_val = np.concatenate(psds[n_train:n_train+n_val])
psds_val = psds_val.reshape(len(psds_val), -1)
ages_val = np.concatenate(ages[n_train:n_train+n_val])

print("# subjects")
print(f"train set: {n_train}")
print(f"validation set: {n_val}")
print(f"test set: {n_test}")

# subjects
train set: 132
validation set: 28
test set: 28


In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(psds_train)

rfg = RandomForestRegressor(n_jobs=128)
rfg.fit(psds_train, ages_train)
r_squared_rfg_train = rfg.score(scaler.transform(psds_train), ages_train)
r_squared_rfg_val = rfg.score(scaler.transform(psds_val), ages_val)

print("Random forest regressor")
print(f"train R²:     {r_squared_rfg_train:.3}")
print(f"val R²:     {r_squared_rfg_val:.3}")

Random forest regressor
train R²:     -0.765
val R²:     -1.35


In [ ]:
# percent_feat = 0.5
# feat_mask = rfg.feature_importances_ > np.percentile(rfg.feature_importances_, 100-percent_feat)
# X_train = X_train[:, feat_mask]
# X_val = X_val[:, feat_mask]
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

gbr = GradientBoostingRegressor(n_estimators=400)
gbr.fit(psds_train, ages_train)
r_squared_gbr_train = gbr.score(scaler.transform(psds_train), ages_train)
r_squared_gbr_val = gbr.score(scaler.transform(psds_val), ages_val)

print("\nTraining performance (R²) \n")
print(f"random forest regressor:     {r_squared_rfg_train:.3}")
print(f"gradient boosting regressor: {r_squared_gbr_train:.3}")
print("\nValidation performance (R²) \n")
print(f"random forest regressor:     {r_squared_rfg_val:.3}")
print(f"gradient boosting regressor: {r_squared_gbr_val:.3}")

### KFold Crossvalidation

In [ ]:
# X, y = psds.reshape(len(psds), -1), ages
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.33)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

#### Random Forest Regressor (sklearn)

In [20]:
rfg = RandomForestRegressor(n_jobs=-1)
rfg_pipe = Pipeline(steps=[("scaler", scaler), ("rfg", rfg)])

cv = KFold(n_splits=10)

r_squared_rfg_val = cross_val_score(rfg_pipe, X=psds_train, y=ages_train, scoring='r2', cv=cv, n_jobs=-1)
r_squared_rfg_val_mean, r_squared_rfg_val_std = r_squared_rfg_val.mean(), r_squared_rfg_val.std()

print(f"{r_squared_rfg_val_mean:.3} +/- {r_squared_rfg_val_std:.3}")

-0.0101 +/- 0.269


#### Gradient Boosting Regressor (sklearn)

Slow to train since it doesn't support parallelism

In [ ]:
gbr = GradientBoostingRegressor()
gbr_pipe = Pipeline(steps=[("scaler", scaler), ("gbr", gbr)])

cv = KFold(n_splits=10)

r_squared_gbr_val = cross_val_score(gbr_pipe, X=psds_train, y=ages_train, scoring='r2', cv=cv, n_jobs=32)
r_squared_gbr_val_mean, r_squared_gbr_val_std = r_squared_gbr_val.mean(), r_squared_gbr_val.std()

print(f"{r_squared_gbr_val_mean:.3} +/- {r_squared_gbr_val_std:.3}")

#### Gradient Boosting Regressor (XGboost)

In [21]:
from xgboost import XGBRegressor

In [22]:
xgbr = XGBRegressor(n_estimators=160, max_depth=2, learning_rate=0.1, tree_method='gpu_hist', gpu_id=0)
xgbr_pipe = Pipeline(steps=[("scaler", scaler), ("xgbr", xgbr)])

cv = KFold(n_splits=10)

r_squared_xgbr_val = cross_val_score(xgbr_pipe, X=psds_train, y=ages_train, scoring='r2', cv=cv, n_jobs=-1)
r_squared_xgbr_val_mean, r_squared_xgbr_val_std = r_squared_xgbr_val.mean(), r_squared_xgbr_val.std()

print(f"{r_squared_xgbr_val_mean:.3} +/- {r_squared_xgbr_val_std:.3}")

0.062 +/- 0.201


### XGBoost hyperparameter tuning (on GPU)

In [ ]:
param_distributions = {
    "xg_gbr__n_estimators": [20, 80, 160, 320, 640, 1280],
    "xg_gbr__max_depth": [2, 3, 4, 5, 6],
    "xg_gbr__learning_rate": [1e-3, 1e-2, 1e-1, 0.2]
}

xgb_regr = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
xgb_pipe = Pipeline(steps=[("scaler", scaler), ("xg_gbr", xgb_regr)])

inner_cv = KFold(n_splits=5)
outer_cv = KFold(n_splits=5)

xgb_regr_cv = RandomizedSearchCV(xgb_pipe, param_distributions, cv=inner_cv, n_iter=25)
search = xgb_regr_cv.fit(psds_train, ages_train)

r_squared_xgbr_train = xgb_regr_cv.best_score_
r_squared_xgbr_val = cross_val_score(xgb_regr_cv, X=psds_train, y=ages_train, scoring='r2', cv=outer_cv)

print(
    r_squared_xgbr_train,
    r_squared_xgbr_val.mean(),
    r_squared_xgbr_val.std()
)